# KNN algorithm and Conformal Prediction


### loading data set from iris and ionosphere

In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import numpy.linalg as la

iris=load_iris()
X=np.genfromtxt("//home/cim/pgt/mhac062/Downloads/ionosphere.txt",delimiter=",",usecols=np.arange(34))
y=np.genfromtxt("//home/cim/pgt/mhac062/Downloads/ionosphere.txt",delimiter=",",usecols=34,dtype='int')


### KNN function build for general (K)

In [328]:
# finding index of sorted array  
def sorting_array(arr):
    n=arr.shape[0]
    sort_index=np.zeros(n)
    count=0
    for j in range(n):
        result = 0
        for i in range(n): 
            if (arr[i] < arr[j]): 
                result += 1
            if (arr[i] == arr[j] and i < j): 
                result += 1
        
        sort_index[count]=result
        count+=1
    return sort_index

# finding k nearest values  
def fetch_k_values(arr,k):
    k_values=np.zeros(k)
    count=0
    for v in range(y_train.shape[0]):
        if arr[0,v,0]<=k-1:
            k_values[count]=arr[0,v,1]
            count+=1
    return k_values


In [304]:
# Knn function definition 
def NNfit(X_train,y_train,X_test,y_test,k):
    import time
    import numpy.linalg as la
    start = time.time()
    y_predict=np.zeros(y_test.shape[0])
    # finding distance between test samples and training set 
    for i in range(X_test.shape[0]):
        my_array=np.zeros(X_train.shape[0])
        for j in range(X_train.shape[0]):
            my_array[j]=la.norm(X_test[i,:]-X_train[j,:])
    # sorting the array and finding the index    
        sorted_array=sorting_array(my_array)
    # zipping sorted array with the labels      
        d=np.dstack((sorted_array,y_train)) 
    # finding the k nearest sample labels      
        max_label_count=fetch_k_values(d,k)
    # voting of lables
        (values,counts) = np.unique(max_label_count,return_counts=True)
        ind=np.argmax(counts)
        y_predict[i]=int(values[ind])     
   
    
    print('\npredicted in: ',"%.2f" %(time.time() - start),"seconds")    
     # finding number of errors 
    print('number of errors: ',y_test.size-np.count_nonzero(y_predict == y_test))
     # finding the error_rate  
    print('error_rate: ',(y_test.size-np.count_nonzero(y_predict == y_test))/y_test.shape[0])
    





predicted in:  3.19 seconds
number of errors:  13
error_rate:  0.14772727272727273


### iris_split

In [8]:
X_train,X_test,y_train,y_test=train_test_split(iris['data'],iris['target'],random_state=503)

###  comparison with  sklearn.neighbors library-iris

In [353]:




print("**** My implementation- Result ****\n")
NNfit(X_train,y_train,X_test,y_test,3)
print("\ns**** klearn.neighbors - Result ****\n")
import time
start = time.time()
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print('\npredicted in: ',"%.2f" %(time.time() - start),"seconds")

print("number of errors: ",y_test.size-np.count_nonzero(y_pred == y_test))

print("test_error_rate: ",1-knn.score(X_test,y_test))

**** My implementation- Result ****


predicted in:  3.14 seconds
number of errors:  13
error_rate:  0.14772727272727273

s**** klearn.neighbors - Result ****


predicted in:  0.00 seconds
number of errors:  13
test_error_rate:  0.147727272727


### splitting inonosphere data

In [349]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=503)


###  comparison with  sklearn.neighbors library-ionosphere

In [352]:


print("**** My implementation- Result **** \n")
NNfit(X_train,y_train,X_test,y_test,3)
print("\n**** sklearn.neighbors - Result **** \n")
import time
start = time.time()
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print('\npredicted in: ',"%.2f" %(time.time() - start),"seconds")

print("number of errors: ",y_test.size-np.count_nonzero(y_pred == y_test))

print("test_error_rate: ",1-knn.score(X_test,y_test))

**** My implementation- Result **** 


predicted in:  3.18 seconds
number of errors:  13
error_rate:  0.14772727272727273

**** sklearn.neighbors - Result **** 


predicted in:  0.00 seconds
number of errors:  13
test_error_rate:  0.147727272727


## conformal prediction 



### function definition - calculation of confirmity score 

In [9]:
# function definition along with parameters
def conformity_score(X_train,y_train,X_test,y_test,i,j,labels):
    
    import math
    # concatenating (X,y) of training set
    x_train_concat=np.concatenate((X_train,y_train[:,None]),axis=1)
    # concatenating (X,y) of one test set per function
    x_test_add=np.concatenate((X_test[i:i+1,],labels[j:j+1,None]),axis=1)
    # concatenating (X,y) of all training set and one test set
    x_train_test_concat=np.concatenate((x_train_concat,x_test_add))
    # concatenating y of training set and one test set
    y_concat=np.concatenate((y_train,labels[j:j+1]))
    # assign size of CF_score array
    CF_score=np.zeros(x_train_test_concat.shape[0])
    # iterate through each (x,y) of set and find distance
    for k in range(x_train_test_concat.shape[0]):
        arr_same=[]
        arr_diff=[]
        for l in range(x_train_test_concat.shape[0]):
            if k!=l:
                # find distance between sample of same class
                if x_train_test_concat[k,-1]==y_concat[l]:
                    arr_same.append(la.norm(x_train_test_concat[k,:-1]-x_train_test_concat[l,:-1]))
                # find distance between sample of different class
                else:
                    arr_diff.append(la.norm(x_train_test_concat[k,:-1]-x_train_test_concat[l,:-1]))
    # exception handling for ZeroDivisionError
    # using formula -(distance between nearest sample of same class)/(distance between nearest sample of diff class)
        try:
            CF_score[k]=(min(arr_diff)/min(arr_same)) 
        except ZeroDivisionError:
            CF_score[k]=np.inf
    return CF_score


### function definition - calculation of P value

In [10]:
# calculating p value of each CF_score array
def p_value(CF_score):
    count=0
    for m in range(CF_score.shape[0]):
    
        if CF_score[m]<=CF_score[-1]:
            count+=1
    p_value=(count/CF_score.shape[0])
    return p_value
import numpy as np
p_value(np.array([1,2,3,6,5]))

0.8

### function definition - calculation of P value for every label

In [14]:
# calculation of p_value of each test sample for every label
def p_value_array(X_train,X_test,y_train,y_test,labels):
    
    labels_array=np.zeros((X_test.shape[0],labels.shape[0]))
    for i in range(X_test.shape[0]):
        for j in range(labels.shape[0]):
            labels_array[i,j]=p_value(conformity_score(X_train,y_train,X_test,y_test,i,j,labels))
    return labels_array 

labels=np.unique(y_train)
p_value_array(X_train,X_test,y_train,y_test,labels)

/usr/lib/python3/dist-packages/ipykernel_launcher.py:30: RuntimeWarning: divide by zero encountered in double_scalars
/usr/lib/python3/dist-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in double_scalars


array([[ 0.68141593,  0.00884956,  0.00884956],
       [ 0.7079646 ,  0.00884956,  0.00884956],
       [ 0.00884956,  0.60176991,  0.00884956],
       [ 0.00884956,  0.6460177 ,  0.00884956],
       [ 0.00884956,  0.00884956,  0.33628319],
       [ 0.59292035,  0.00884956,  0.00884956],
       [ 0.00884956,  0.00884956,  0.08849558],
       [ 0.00884956,  0.00884956,  0.69911504],
       [ 0.00884956,  0.00884956,  0.33628319],
       [ 0.00884956,  0.44247788,  0.00884956],
       [ 0.79646018,  0.00884956,  0.00884956],
       [ 0.84955752,  0.00884956,  0.00884956],
       [ 0.00884956,  0.00884956,  0.17699115],
       [ 0.00884956,  0.42477876,  0.00884956],
       [ 0.75221239,  0.00884956,  0.00884956],
       [ 0.00884956,  0.00884956,  0.08849558],
       [ 0.00884956,  0.00884956,  0.2920354 ],
       [ 0.78761062,  0.00884956,  0.00884956],
       [ 0.00884956,  0.30088496,  0.00884956],
       [ 0.00884956,  0.44247788,  0.00884956],
       [ 0.00884956,  0.02654867,  0.079

### function definition - calculation of average false P value 

In [12]:
# calculation of average false p_value
def average_false_pvalue(X_train,X_test,y_train,y_test):
    
    labels=np.unique(y_train)
    p_value_array1=p_value_array(X_train,X_test,y_train,y_test,labels)
    sum=0
    for i in range(p_value_array1.shape[0]):
        for j in range(labels.shape[0]):
            # summation of false_pvalues 
            if labels[j]!=y_test[i]:
                sum+=p_value_array1[i,j]
    # average of false_p_value
    return (sum/(X_test.shape[0]*(labels.shape[0]-1))) 


### calculation of average false P value of iris dataset

In [362]:
import time
start = time.time()
X_train,X_test,y_train,y_test=train_test_split(iris['data'],iris['target'],random_state=503)
print('Average false P_value of iris data set is ',average_false_pvalue(X_train,X_test,y_train,y_test))
print('\ncalculated in: ',"%.2f" %(time.time() - start),"seconds")

/usr/lib/python3/dist-packages/ipykernel_launcher.py:30: RuntimeWarning: divide by zero encountered in double_scalars
/usr/lib/python3/dist-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in double_scalars


Average false P_value of iris data set is  0.0109455053563

calculated in:  8.28 seconds


### calculation of average false P value of ionosphere dataset

In [315]:
import time
start = time.time()
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=503)
print('Average false P_value of ionoshphere data set is ',average_false_pvalue(X_train,X_test,y_train,y_test))
print('\calculated in: ',"%.2f" %(time.time() - start),"seconds")

/usr/lib/python3/dist-packages/ipykernel_launcher.py:30: RuntimeWarning: divide by zero encountered in double_scalars


Average false P_value of iris data set is  0.0636191460055

predicted in:  72.15 seconds
